In [1]:
# set environment and import packages
import os
from pandas import read_csv
import pandas as pd
import geopandas as gps
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
import constants as c

os.chdir('/Users/xiaodanxu/Documents/SynthFirm.nosync/mode_choice')



In [2]:
sctg_group_lookup = read_csv(c.param_dir + c.sctg_group_lookup_file, sep = ',')
mesozone_lookup = read_csv(c.param_dir + c.mesozone_id_lookup_file, sep = ',')

truck_mode = ['For-hire Truck', 'Private Truck']
# max_ton_lookup = {'sctg1': 218.327, 'sctg2': 51.13091, 'sctg3': 94.56035, 'sctg4': 4.4861, 'sctg5': 51.64694}

In [6]:
combined_modeled_OD = None
mode_choide_by_commodity = None
combined_truck_output = None
for sctg in c.list_of_sctg_group:
    print(sctg)
    b2b_dir = c.output_dir + sctg
    list_of_b2b_files = listdir(b2b_dir)
    iterator = 0
#     cut_off_point = max_ton_lookup[sctg] # shipment capacity for this shipment
    for file in list_of_b2b_files:
        if file == '.DS_Store':
            continue
        if iterator%10 == 0:
            print(iterator)
        modeled_OD_by_sctg = read_csv(b2b_dir + '/' + file, sep = ',')
#         print(modeled_OD_by_sctg.columns)
        list_of_var = ['BuyerID', 'BuyerZone', 'BuyerNAICS', 'SellerID',
           'SellerZone', 'SellerNAICS', 'TruckLoad', 'SCTG_Group', 
           'shipment_id', 'orig_FAFID', 'dest_FAFID', 
           'mode_choice', 'probability', 'Distance', 'Travel_time']
#         print(modeled_OD_by_sctg.head(5))
        truck_output = modeled_OD_by_sctg.loc[modeled_OD_by_sctg['mode_choice'].isin(truck_mode), list_of_var]
        int_var = ['BuyerID', 'BuyerZone', 'SellerID',
           'SellerZone', 'SCTG_Group', 
           'shipment_id', 'orig_FAFID', 'dest_FAFID']
        truck_output.loc[:, int_var] = truck_output.loc[:, int_var].astype(int)
        truck_output.to_csv(c.output_dir + sctg + '_truck/truck_only_OD_' + sctg + '_' + 
                            str(iterator) + '.csv', index = False)
#         if len(truck_output) > 0:
#             combined_truck_output = pd.concat([combined_truck_output, truck_output])
        # process aggregated shipment count and tonmile for CFS
#         mode_choice_aggregated = modeled_OD_by_sctg.groupby(['mode_choice', 'SCTG_Group', 'in_study_area'])[['shipment_id']].count()
#         mode_choice_aggregated = mode_choice_aggregated.reset_index()
#         modeled_OD_by_sctg.loc[:, 'tonmile'] = modeled_OD_by_sctg.loc[:, 'TruckLoad'] * modeled_OD_by_sctg.loc[:, 'Distance']
#         distance_and_weight_aggregated = modeled_OD_by_sctg.groupby(['mode_choice', 'SCTG_Group', 'in_study_area'])[['TruckLoad', 'tonmile']].sum()

        
#         mode_choice_aggregated = pd.merge(mode_choice_aggregated, distance_and_weight_aggregated, 
#                                           on = ['mode_choice', 'SCTG_Group', 'in_study_area'], how = 'left')
#         mode_choice_aggregated.loc[:, 'SCTG_Group'] = c.sctg_def[sctg]
#         mode_choice_aggregated.loc[:, 'chunk_id'] = iterator
#         mode_choide_by_commodity = pd.concat([mode_choide_by_commodity, mode_choice_aggregated])
        
        ## compute national shipment count and tonmile
        modeled_OD_by_sctg['ShipmentLoad'] = modeled_OD_by_sctg['TruckLoad'] / 1000 # convert to thousand tons
        
        #print(cut_off_point)
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['ShipmentLoad'] > cut_off_point, 'ShipmentLoad'] = cut_off_point
        modeled_OD_by_sctg['tmiles'] = modeled_OD_by_sctg['ShipmentLoad'] * 1000 * modeled_OD_by_sctg['Distance']
        modeled_OD_by_sctg = pd.merge(modeled_OD_by_sctg, mesozone_lookup, 
                                      left_on = ['SellerZone', 'orig_FAFID'], 
                                    right_on = ['MESOZONE', 'FAFID'], how = 'left')
        modeled_OD_by_sctg = modeled_OD_by_sctg.rename(columns={"GEOID": "orig_GEOID",
                                                                "CBPZONE": "orig_CBPZONE", 
                                                                "MESOZONE":"orig_MESOZONE", 
                                                                "FAFNAME":"orig_FAFNAME"})
        modeled_OD_by_sctg = pd.merge(modeled_OD_by_sctg, mesozone_lookup, 
                                      left_on = ['BuyerZone', 'dest_FAFID'], 
                                    right_on = ['MESOZONE', 'FAFID'], how = 'left')
        modeled_OD_by_sctg = modeled_OD_by_sctg.rename(columns={"GEOID": "dest_GEOID", 
                                                                "CBPZONE": "dest_CBPZONE", 
                                                                "MESOZONE":"dest_MESOZONE", 
                                                               "FAFNAME":"dest_FAFNAME"})    
        agg_OD_by_sctg = modeled_OD_by_sctg.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'])[['tmiles', 'ShipmentLoad']].sum()        
        agg_OD_by_sctg = agg_OD_by_sctg.reset_index()
        agg_count_by_sctg = modeled_OD_by_sctg.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'])[['shipment_id']].count() 
        agg_count_by_sctg = agg_count_by_sctg.reset_index()
        agg_OD_by_sctg = pd.merge(agg_OD_by_sctg, agg_count_by_sctg, 
                                  on = ["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'],
                                  how = 'left')
        agg_OD_by_sctg = agg_OD_by_sctg.rename(columns={"shipment_id": "count"})
        agg_OD_by_sctg.loc[:, 'SCTG_Name'] = c.sctg_def[sctg]
        agg_OD_by_sctg.loc[:, 'chunk_id'] = iterator
        combined_modeled_OD = pd.concat([combined_modeled_OD, agg_OD_by_sctg], sort = False)
        iterator += 1 
#         break        
#     break
#     combined_truck_output.to_csv(c.input_dir + 'truck_only_OD_' + sctg + '.csv', index = False)
# combined_modeled_OD = pd.merge(combined_modeled_OD, sctg_group_definition, on = ['SCTG_Group'], how = 'left')
combined_modeled_OD.head(10)    

sctg1
0
10
sctg2
0
10
sctg3
0


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


sctg4
0
10
20
sctg5
0


,orig_FAFID,orig_FAFNAME,dest_FAFID,dest_FAFNAME,SCTG_Group,mode_choice,tmiles,ShipmentLoad,count,SCTG_Name,chunk_id
0,50,Arkansas,481,Austin,1,For-hire Truck,3.573676e+08,627.742657,13099,bulk,0
1,50,Arkansas,481,Austin,1,Rail/IMX,2.362758e+08,240.237724,5013,bulk,0
2,229,Rest of LA,488,San Antonio,1,For-hire Truck,4.136916e+08,390.876187,8156,bulk,0
3,229,Rest of LA,488,San Antonio,1,Rail/IMX,4.335691e+08,350.427462,7312,bulk,0
4,481,Austin,481,Austin,1,For-hire Truck,0.000000e+00,225.593747,4480,bulk,0
5,481,Austin,481,Austin,1,Private Truck,0.000000e+00,450.221373,8895,bulk,0
6,481,Austin,481,Austin,1,Rail/IMX,0.000000e+00,13.206317,260,bulk,0
7,481,Austin,487,Laredo,1,For-hire Truck,2.569788e+06,11.006932,230,bulk,0
8,481,Austin,487,Laredo,1,Private Truck,1.029931e+06,4.411407,93,bulk,0
9,481,Austin,487,Laredo,1,Rail/IMX,4.360680e+05,1.724785,36,bulk,0


In [8]:
combined_modeled_OD_agg = combined_modeled_OD.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", \
                                                       "dest_FAFNAME", "SCTG_Group", 'SCTG_Name',
                                                       'mode_choice'])[['tmiles', 'ShipmentLoad', 'count']].sum()
combined_modeled_OD_agg = combined_modeled_OD_agg.reset_index()
combined_modeled_OD_agg.head(5)
# combined_modeled_OD_agg.loc[:, 'in_study_area'] = 0
# buffer = combined_modeled_OD_agg.loc[:, 'orig_FAFID'].isin(c.bay_area_region_code) | \
#         combined_modeled_OD_agg.loc[:, 'dest_FAFID'].isin(c.bay_area_region_code)
# combined_modeled_OD_agg.loc[buffer, 'in_study_area'] = 1

combined_modeled_OD_agg.loc[:, 'outbound'] = 0
combined_modeled_OD_agg.loc[combined_modeled_OD_agg.loc[:, 'orig_FAFID'].isin(c.region_code), 'outbound'] = 1

combined_modeled_OD_agg.loc[:, 'inbound'] = 0
combined_modeled_OD_agg.loc[combined_modeled_OD_agg.loc[:, 'dest_FAFID'].isin(c.region_code), 'inbound'] = 1

combined_modeled_OD_agg.loc[:, 'orig_FAFID'] = combined_modeled_OD_agg.loc[:, 'orig_FAFID'].astype(int)
combined_modeled_OD_agg.loc[:, 'dest_FAFID'] = combined_modeled_OD_agg.loc[:, 'dest_FAFID'].astype(int)
combined_modeled_OD_agg.loc[:, 'SCTG_Group'] = combined_modeled_OD_agg.loc[:, 'SCTG_Group'].astype(int)
combined_modeled_OD_agg.loc[:, 'Distance'] = combined_modeled_OD_agg.loc[:, 'tmiles'] / 1000 / combined_modeled_OD_agg.loc[:, 'ShipmentLoad']
combined_modeled_OD_agg.head(10)

,orig_FAFID,orig_FAFNAME,dest_FAFID,dest_FAFNAME,SCTG_Group,SCTG_Name,mode_choice,tmiles,ShipmentLoad,count,outbound,inbound,Distance
0,11,Birmingham,481,Austin,4,mfr_goods,For-hire Truck,2.624185e+05,0.305935,101,0,1,857.760000
1,11,Birmingham,481,Austin,4,mfr_goods,Rail/IMX,5.992943e+03,0.006023,2,0,1,994.940199
2,11,Birmingham,488,San Antonio,1,bulk,For-hire Truck,4.509948e+07,47.624537,993,0,1,946.980000
3,11,Birmingham,488,San Antonio,1,bulk,Rail/IMX,1.578538e+07,17.313653,361,0,1,911.730199
4,11,Birmingham,488,San Antonio,4,mfr_goods,For-hire Truck,8.880679e+05,0.937789,313,0,1,946.980000
5,11,Birmingham,488,San Antonio,4,mfr_goods,Rail/IMX,1.089795e+04,0.011953,4,0,1,911.730199
6,11,Birmingham,489,Rest of TX,4,mfr_goods,Air,9.217009e+03,0.011993,4,0,1,768.510811
7,11,Birmingham,489,Rest of TX,4,mfr_goods,For-hire Truck,6.140562e+06,6.923156,2308,0,1,886.960000
8,11,Birmingham,489,Rest of TX,4,mfr_goods,Rail/IMX,4.134362e+04,0.044987,15,0,1,919.015760
9,12,Mobile,481,Austin,4,mfr_goods,For-hire Truck,1.270590e+06,1.385610,464,0,1,916.990000


In [10]:
combined_modeled_OD_agg.to_csv(c.output_dir + 'processed_b2b_flow_summary.csv', sep = ',')